<img src="misc/DHR-Health-Logo.png" width="50%">

### **Business Intelligence**

# py835

The **py835** Python package provides a robust toolset for parsing EDI 835 files using the `pyx12` library. It processes healthcare claim information from EDI 835 files into structured formats like Pandas DataFrames and JSON for seamless data manipulation, reporting, and analysis. This allows organizations to quickly extract their data from these files for long-term storage and analysis.

Note that this project is still very much in the early stages. If you require a stable version, please fork this Github repository. We are working on providing a way of translating fields, such as error and reason codes, though that functionality is in the works. 

#### **Table of Contents**
- [Features](#features)
- [Installation](#installation)
- [Structure of an 835 file](#structure-of-an-835-file)
- [Usage](#usage)
  - [Flattening DataFrames (long-to-wide)](#flattening-dataframes)
  - [Accessing 835 Components](#accessing-835-components)
- [Data Tree](#data-tree)
- [Pandas DataFrames](#pandas-dataframes)
- [Quick Export](#quick-export)
- [Contributing](#contributing)
- [License](#license)

## Features

- **Parse EDI 835 Files:** Load and process `.835` EDI files for healthcare claims and payment information.
- **Extract Data:** Extracts detailed information, including functional groups, transaction sets, claims, services, adjustments, and references.
- **DataFrame Output:** Organizes parsed data into Pandas DataFrames for more convenient analysis.
- **Column Renaming:** Automatically renames columns based on EDI segment codes and descriptions for better readability.
- **Pivot Tables:** Supports pivoting data (e.g., CAS and REF segments) for deeper analysis.
- **JSON Export:** Supports exporting parsed data to JSON format (via pandas) for further use in other systems.


## Installation

To install this package, run the following command:

```bash
pip install git+https://github.com/DHR-Health/py835.git
```

### Dependencies

- `pyx12`: Python library for EDI file parsing.
- `pandas`: Used for organizing parsed data into DataFrames.
- `io`: Standard Python module for handling input/output operations.
- `json`: Used for exporting data to JSON format.

<figcaption><h3>Structure of an 835 file</h3></figcaption>
<img src="misc\835 Structure.png">

# Usage
Parse an 835 file using `py835` to access the data as Pandas dataframes. The parser systematically breaks down the 835 data into hierarchical layers, reflecting the structure of the EDI 835 file. For example, to access statements within the file:


In [1]:
import py835

# Initialize the parser with the path to your EDI file
parser = py835.Parser(r'misc\example.835')
parser.TABLES['STATEMENTS']

,header_id,functional_group_id,statement_id,segment,field,name,value
0,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,ST,ST01,Transaction Set Identifier Code,835
1,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,ST,ST02,Transaction Set Control Number,35681
2,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,ST,ST03,Implementation Convention Reference,None


The image below shows the relationships between the tables accessible via `parser.pandas` (if it exists). You can find out which tables are available using the dictionary keys:

In [2]:
[key for key in parser.TABLES]

['HEADER',
 'FUNCTIONAL_GROUPS',
 'STATEMENTS',
 'STATEMENTS_REF',
 'CLAIMS',
 'SERVICES',
 'SERVICES_CAS',
 'SERVICES_DTM',
 'FOOTER']


<img src="misc\tree_structure.png">

By default, **py835** dataframes are in long-format. The dataframes generated by the package include a custom method for converting these tables to wide format. Use `.flatten` to do so:

In [3]:
parser.TABLES['STATEMENTS_REF'].flatten().head(3)

,header_id,functional_group_id,statement_id,ref_id,REFEV REF01,REFEV REF02,REFEV REF03,REFEV REF04,REFTJ REF01,REFTJ REF02,REFTJ REF03,REFTJ REF04
0,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,ncuqDP-ZcDx7m-l1RZyq-CxivmO,EV,XYZ CLEARINGHOUSE,NaN,NaN,NaN,NaN,NaN,NaN
1,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,XvQOI3-PUWxdk-MP8vGQ-jT2tfs,NaN,NaN,NaN,NaN,TJ,212121212,NaN,NaN


You can add descriptions for the fields while flattening the data as well (`descriptions = True`) and add a prefix to the flattened column names (`prefix = <PREFIX>`)

In [4]:
parser.TABLES['STATEMENTS_REF'].head(5).flatten(prefix='My Prefix ',descriptions = True)

,header_id,functional_group_id,statement_id,ref_id,My Prefix REFEV REF01 Reference Identification Qualifier,My Prefix REFEV REF02 Receiver Identifier,My Prefix REFEV REF03 Description,My Prefix REFEV REF04 Reference Identifier,My Prefix REFTJ REF01 Reference Identification Qualifier
0,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,ncuqDP-ZcDx7m-l1RZyq-CxivmO,EV,XYZ CLEARINGHOUSE,NaN,NaN,NaN
1,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,XvQOI3-PUWxdk-MP8vGQ-jT2tfs,NaN,NaN,NaN,NaN,TJ


You can access individual components of the 835 file using the dictionary at `parser.dict`, if you so desire. The `parser.dict` dictionary contains your 835 data as a JSON:

In [5]:
# Get the STATEMENTS_REF table as a JSON
parser.dict['STATEMENTS_REF']

[{'header_id': '38JAXc-MQtvih-L0oNNm-c4k919',
  'functional_group_id': 'jlfX4a-WoXJRj-Tn76iW-O6qyNg',
  'statement_id': 'oiHc1Y-H46TSu-FtcC2R-HzfUXe',
  'ref_id': 'ncuqDP-ZcDx7m-l1RZyq-CxivmO',
  'segments': [{'segment': 'REFEV',
    'REF01': {'name': 'Reference Identification Qualifier', 'value': 'EV'},
    'REF02': {'name': 'Receiver Identifier', 'value': 'XYZ CLEARINGHOUSE'},
    'REF03': {'name': 'Description', 'value': None},
    'REF04': {'name': 'Reference Identifier', 'value': None}}]},
 {'header_id': '38JAXc-MQtvih-L0oNNm-c4k919',
  'functional_group_id': 'jlfX4a-WoXJRj-Tn76iW-O6qyNg',
  'statement_id': 'oiHc1Y-H46TSu-FtcC2R-HzfUXe',
  'ref_id': 'XvQOI3-PUWxdk-MP8vGQ-jT2tfs',
  'segments': [{'segment': 'REFTJ',
    'REF01': {'name': 'Reference Identification Qualifier', 'value': 'TJ'},
    'REF02': {'name': 'Additional Payee Identifier', 'value': '212121212'},
    'REF03': {'name': 'Description', 'value': None},
    'REF04': {'name': 'Reference Identifier', 'value': None}}]}]

# Pandas Dataframes
The **py835** parser generates pandas dataframes from your 835 file so that you can quickly import the data into your data warehouse. These are available using the `parser.TABLES` dictionary. The parser generates ids (`header_id`, `functional_group_id`, `statement_id`, `claim_id`, and `service_id`) as it passes through each component. You can use these IDs when joining the various tables together.

1. **ISA (Interchange Control Header):**  `parser.TABLES['HEADER']`
   
   The top-level layer is the ISA segment, also called The Header, which contains metadata about the interchange, such as sender/receiver information, control numbers, and transaction timestamps. This segment serves as a unique identifier for the file. You can retrieve the ISA header as a Pandas DataFrame using `parser.pandas['HEADER']`. This allows for easy analysis of interchange metadata, including file-level information.


In [6]:
# Example Header
parser.TABLES['HEADER'].head(5)

,header_id,segment,field,name,value
0,38JAXc-MQtvih-L0oNNm-c4k919,ISA,ISA01,Authorization Information Qualifier,00
1,38JAXc-MQtvih-L0oNNm-c4k919,ISA,ISA02,Authorization Information,
2,38JAXc-MQtvih-L0oNNm-c4k919,ISA,ISA03,Security Information Qualifier,00
3,38JAXc-MQtvih-L0oNNm-c4k919,ISA,ISA04,Security Information,
4,38JAXc-MQtvih-L0oNNm-c4k919,ISA,ISA05,Interchange Sender ID Qualifier,ZZ



2. **Functional Groups (GS):**  `parser.TABLES['FUNCTIONAL_GROUPS']`
   
   Within each `ISA` segment, there are one or more `GS` (Functional Group Header) segments. Functional groups organize related transaction sets under a specific purpose or business function, such as claims, remittance advice, or payment acknowledgments. You can retrieve information about the functional groups as a Pandas DataFrame using `parser.TABLES['FUNCTIONAL_GROUPS']`. This table can be joined with the ISA table on the `'header_id'` column for comprehensive data analysis across files.


In [7]:
# Example Functional Group
parser.TABLES['FUNCTIONAL_GROUPS'].head(5)

,header_id,functional_group_id,segment,field,name,value
0,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,GS,GS01,Functional Identifier Code,HP
1,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,GS,GS02,Application Sender's Code,ABCD
2,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,GS,GS03,Application Receiver's Code,ABCD
3,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,GS,GS04,Date,20190827
4,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,GS,GS05,Time,12345678



1. **Statements (ST):  parser.TABLES['STATEMENTS']**
   
   Inside each functional group, `ST` segments define statements, also known as transaction sets. Each transaction set corresponds to a statement, bundling related claims, payments, or service details. One 835 file can have multiple transaction sets, which serve as logical groups for payment and claim details. You can extract statement data as a Pandas DataFrame using `parser.TABLES['STATEMENTS']`. This table can be joined with the functional group data using the composite key `['header_id', 'functional_group_id']`.



4. **Claims (CLP):**  `parser.TABLES['CLAIMS']`

   Each transaction set breaks down further into individual claims (`CLP` segments). Claims represent billing information for healthcare services rendered, including important details such as claim IDs, patient identifiers, the total amount billed, adjustments, payments made, and any denials or rejections. You can retrieve claim information as a Pandas DataFrame using `parser.TABLES['CLAIMS']`. Claims can be joined to statement data using the composite key `['header_id', 'functional_group_id', 'statement_id']`.

    4a. **Claim Adjustments (CAS):**  `parser.TABLES['CLAIMS_CAS']`

       Claims often have adjustments (`CAS` segments), which represent reductions or additions to the claim amount based on specific reasons like contractual obligations, patient responsibility, or denials. The parser extracts all adjustments, grouping them by claim, and allows you to retrieve this data in a Pandas DataFrame via `parser.TABLES['CLAIMS_CAS']`.

    4b. **Claim References (REF):**  `parser.TABLES['CLAIMS_REF']`

       The parser captures `REF` (Reference Identification) segments, which contain additional reference information related to claims. These may include provider identification numbers, patient account numbers, or other important reference codes. You can access reference data as a Pandas DataFrame via `parser.TABLES['CLAIMS_REF']`.

    4c. **Claim Service Identification (LQ):**  `parser.TABLES['CLAIMS_LQ']`

       The `LQ` segments provide additional information related to the services or claims, such as service qualifiers and codes. These segments are extracted into a Pandas DataFrame for claims via `parser.TABLES['CLAIMS_LQ']`, which can be joined to the claims table.

    4d. **Claim Date/Time (DTM):**  `parser.TABLES['CLAIMS_DTM']`
    
       The `DTM` segments represent various date and time-related information for claims (e.g., service dates, adjudication dates). You can retrieve claim-related date/time information as a Pandas DataFrame via `parser.TABLES['CLAIMS_DTM']`.

In [8]:
# Example Claim Data
parser.TABLES['CLAIMS'].head(5)

,header_id,functional_group_id,statement_id,claim_id,segment,field,name,value
0,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,CLP,CLP01,Patient Control Number,7722337
1,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,CLP,CLP02,Claim Status Code,1
2,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,CLP,CLP03,Total Claim Charge Amount,226
3,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,CLP,CLP04,Claim Payment Amount,132
4,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,CLP,CLP05,Patient Responsibility Amount,



5. **Service Line Items (SVC):**  `parser.TABLES['SERVICES']`
   
   Within each claim, service line items (`SVC` segments) detail individual healthcare services or procedures performed during the treatment. The line item data includes service codes, charges, allowed amounts, and any related adjustments. You can extract service line data as a Pandas DataFrame using `parser.TABLES['SERVICES']`. These can be linked to the claims table using the composite key `['header_id', 'functional_group_id', 'statement_id', 'claim_id']`.

    5a. **Service Adjustments (CAS):**  `parser.TABLES['SERVICES_CAS']`

       Services often have adjustments (`CAS` segments), which represent reductions or additions to the service amount based on specific reasons like contractual obligations, patient responsibility, or denials. The parser extracts all adjustments, grouping them by service level, and allows you to retrieve this data as a Pandas DataFrame via `parser.TABLES['SERVICES_CAS']`.

    5b. **Service References (REF):**  `parser.TABLES['SERVICES_REF']`

       The parser captures `REF` (Reference Identification) segments for service-level items, which may contain reference information like procedure codes or authorization numbers. These references are extracted into a Pandas DataFrame using `parser.TABLES['SERVICES_REF']`.

    5c. **Service Identification (LQ):**  `parser.TABLES['SERVICES_LQ']`

       Service-level `LQ` segments contain service-specific qualifiers and codes. You can retrieve this information as a Pandas DataFrame via `parser.TABLES['SERVICES_LQ']`.

    5d. **Service Date/Time (DTM):**  `parser.TABLES['SERVICES_DTM']`
    
       The `DTM` segments for services capture date and time-related information (e.g., service dates, procedure dates). This data is available as a Pandas DataFrame using `parser.TABLES['SERVICES_DTM']`.


In [9]:
# Example Service CAS Data
parser.TABLES['SERVICES_CAS'].head(3)

,header_id,functional_group_id,statement_id,claim_id,service_id,cas_id,segment,field,name,value
0,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,a3T4Db-W8d4Hh-QBceQQ-6jvvYp,AEKWqj-iKhU7Z-6XodJw-skv2I9,CASCO,CAS01,Claim Adjustment Group Code,CO
1,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,a3T4Db-W8d4Hh-QBceQQ-6jvvYp,AEKWqj-iKhU7Z-6XodJw-skv2I9,CASCO,CAS02,Adjustment Reason Code,45
2,38JAXc-MQtvih-L0oNNm-c4k919,jlfX4a-WoXJRj-Tn76iW-O6qyNg,oiHc1Y-H46TSu-FtcC2R-HzfUXe,RROMER-GWObLE-khTPuS-SixYQ1,a3T4Db-W8d4Hh-QBceQQ-6jvvYp,AEKWqj-iKhU7Z-6XodJw-skv2I9,CASCO,CAS03,Adjustment Amount,21



The parser ensures that all segments (ISA, GS, ST, CLP, SVC) are organized in a structured, hierarchical format for easy access and analysis. It also captures important references and adjustments at various levels using `REF` and `CAS` segments, further enhancing the breakdown of claims and services.

# Quick Export
You can use the tables in `parser.pandas` to import the data into your data warehouse for long-term storage, though their long-format structure isn't well-suited for analytics. We've included a way of quickly exporting your data. 


In [10]:
from py835 import Parser

# Initialize the parser with the path to your EDI 835 file
parser = Parser(r'misc\example.835')

bare_bones = parser.flatten(prefix = None,table_names=False,descriptions=False)
bare_bones.head(3)

,header_id,ISA ISA01,ISA ISA02,ISA ISA03,ISA ISA04,ISA ISA05,ISA ISA06,ISA ISA07,ISA ISA08,ISA ISA09,...,CASPR CAS17,CASPR CAS18,CASPR CAS19,dtm_id,DTM472 DTM01,DTM472 DTM02,DTM472 DTM03,DTM472 DTM04,DTM472 DTM05,DTM472 DTM06
0,WpYRlz-kSro93-n6Ewyz-F2erQG,00,,00,,ZZ,ABCPAYER,ZZ,ABCPAYER,190827,...,NaN,NaN,NaN,MwIPQw-T54I26-UGJtgs-6Qn73J,472,20190324,None,None,None,None
1,WpYRlz-kSro93-n6Ewyz-F2erQG,00,,00,,ZZ,ABCPAYER,ZZ,ABCPAYER,190827,...,NaN,NaN,NaN,FaDDFi-MkCydK-q182Gp-6atCcg,472,20190324,None,None,None,None
2,WpYRlz-kSro93-n6Ewyz-F2erQG,00,,00,,ZZ,ABCPAYER,ZZ,ABCPAYER,190827,...,NaN,NaN,NaN,nwaCyY-ki3BBi-vhlz0I-02Yjr5,472,20190324,None,None,None,None



Note that the resulting dataframe can have over 200 columns, depending on the amount of data in your 835 file. There are a few customization options available with the `.flatten` method:
    * `prefix (string|None)` - Add a prefix to every column.
    * `table_names (True|False)` - Prepend the name of the table in the column name.
    * `descriptions (True|False)` - Provide descriptions of the fields in the column name.

In [11]:
from py835 import Parser

# Initialize the parser with the path to your EDI 835 file
parser = Parser(r'misc\example.835')

pretty = parser.flatten(prefix = None,table_names=True,descriptions=True)
pretty.head(3)

,header_id,HEADER ISA ISA01 Authorization Information Qualifier,HEADER ISA ISA02 Authorization Information,HEADER ISA ISA03 Security Information Qualifier,HEADER ISA ISA04 Security Information,HEADER ISA ISA05 Interchange Sender ID Qualifier,HEADER ISA ISA06 Interchange Sender ID,HEADER ISA ISA07 Interchange Receiver ID Qualifier,HEADER ISA ISA08 Interchange Receiver ID,HEADER ISA ISA09 Interchange Date,...,SERVICES_CAS CASPR CAS17 Adjustment Reason Code,SERVICES_CAS CASPR CAS18 Adjustment Amount,SERVICES_CAS CASPR CAS19 Adjustment Quantity,dtm_id,SERVICES_DTM DTM472 DTM01 Date Time Qualifier,SERVICES_DTM DTM472 DTM02 Service Date,SERVICES_DTM DTM472 DTM03 Time,SERVICES_DTM DTM472 DTM04 Time Code,SERVICES_DTM DTM472 DTM05 Date Time Period Format Qualifier,SERVICES_DTM DTM472 DTM06 Date Time Period
0,SVUTMq-wn3wSR-grUQCl-kf683v,00,,00,,ZZ,ABCPAYER,ZZ,ABCPAYER,190827,...,NaN,NaN,NaN,jGHydx-zPptly-wFKzhW-VXhqk2,472,20190324,None,None,None,None
1,SVUTMq-wn3wSR-grUQCl-kf683v,00,,00,,ZZ,ABCPAYER,ZZ,ABCPAYER,190827,...,NaN,NaN,NaN,JYxxJp-aVmwEK-N9m59M-eXuwyW,472,20190324,None,None,None,None
2,SVUTMq-wn3wSR-grUQCl-kf683v,00,,00,,ZZ,ABCPAYER,ZZ,ABCPAYER,190827,...,NaN,NaN,NaN,uTXs8f-zd5F0g-e6N4ro-VKAuEC,472,20190324,None,None,None,None




## Contributing

Contributions are welcome! Feel free to submit pull requests or open issues.

1. Fork the repo.
2. Create your feature branch (`git checkout -b feature/my-feature`).
3. Commit your changes (`git commit -am 'Add some feature'`).
4. Push to the branch (`git push origin feature/my-feature`).
5. Open a pull request.

## License

This project is licensed under the MIT License - see the [LICENSE](LICENSE) file for details.
